# Modellierung der Heterogenität im Titanic-Datensatz

Wie wurde das Prinzip "Frauen und Kinder zuerst" während des Titanic-Unglücks umgesetzt? Gab es eine klare Präferenz für Frauen oder ein komplexeres Bild? Und welche Rolle spielten soziale Schichten bei den Überlebenschancen von Frauen? Ebenso, wie unterschieden sich die Überlebenschancen zwischen Passagieren verschiedener Nationalitäten wie Engländern, Franzosen und Iren? 

Der Titanic-Datensatz enthält verschiedene Variablen, die Informationen über die Passagiere des Schiffes Titanic erfassen. Hier ist eine Beschreibung der variablen Variablen im Trainingsdatensatz des Titanic-Datensatzes:

- $PassengerId$: Eine eindeutige Identifikationsnummer für jeden Passagier.
- $Survived$: Dummy ob der Passagier überlebt hat (1) oder nicht (0).
- $Pclass$: Die Passagierklasse, in der der Passagier gereist ist (1 = Erste Klasse, 2 = Zweite Klasse, 3 = Dritte Klasse).
- $Name$: Der Name des Passagiers.
- $Sex$: Das Geschlecht des Passagiers (male/female).
- $Age$: Das Alter des Passagiers. 
- $SibSp$: Die Anzahl der Geschwister/Ehepartner an Bord.
- $Parch$: Die Anzahl der Eltern/Kinder an Bord.
- $Ticket$: Die Ticketnummer des Passagiers.
- $Fare$: Der gezahlte Fahrpreis.
- $Cabin$: Die Kabinennummer des Passagiers. 
- $Embarked$: Der Hafen, an dem der Passagier eingestiegen ist (C = Cherbourg, Q = Queenstown, S = Southampton).

Es ist wichtig zu beachten, dass der Datensatz einige fehlende Werte für die Variablen $Age$ und $Cabin$ enthält. Ausserdem enthält der Datensatz nicht alle Passagiere, sondern nur 891 Passagiere.

### Laden des Datensatzes

In [1]:
# Laden des Titanic-Datensatzes
library(titanic)
data("titanic_train")  # Lädt den Trainingsdatensatz

# Anzeigen der ersten paar Zeilen des Datensatzes
head(titanic_train)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
,<int>,<int>,<int>,<chr>,<chr>,<dbl>,<int>,<int>,<chr>,<dbl>,<chr>,<chr>
1,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,,S
2,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.2833,C85,C
3,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,,S
4,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
5,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,,S
6,6,0,3,"Moran, Mr. James",male,NA,0,0,330877,8.4583,,Q


### Kodierung qualitativer Variablen

Betrachten wir zunächst die Variable $Embarked$. Cherbour liegt in Frankreich, Queenstown in Irland und Southampton in England.

In [2]:
# Untersuchung der Häufigkeit der Einschiffungsorte
# C = Cherbourg, Q = Queenstown, S = Southampton
table(titanic_train$Embarked)


      C   Q   S 
  2 168  77 644 

Die meisten Passagiere stiegen in Southampton zu, weshalb wir diesen Hafen als Referenzkategorie verwenden. Im Folgenden generieren wir Dummy-Variablen für Cherbour (FR) und Queenstown (IR).

In [3]:
# Erstellung von Dummy-Variablen für Einschiffungsorte (C = Cherbourg, Q = Queenstown, S = Southampton)
titanic_train$FR <- ifelse(titanic_train$Embarked == 'C', 1, 0)
titanic_train$IR <- ifelse(titanic_train$Embarked == 'Q', 1, 0)
table(titanic_train$FR)
table(titanic_train$IR)


  0   1 
723 168 


  0   1 
814  77 

Betrachten wir nun die Kodierung der Variable $Sex$.

In [4]:
# Untersuchung der Geschlechterverteilung
table(titanic_train$Sex)


female   male 
   314    577 

Wir generieren einen Dummy für Frauen.

In [5]:
# Erstellung einer Dummy-Variable für das Geschlecht (1 für Frau, 0 für Mann)
titanic_train$Woman <- ifelse(titanic_train$Sex == 'female', 1, 0)
table(titanic_train$Woman)


  0   1 
577 314 

### Restringierte Regression

Wir beginnen mit der Schätzung unseres Basismodells, das nur den bezahlten Fahrpreis und das Abfahrtsland berücksichtigt.

In [6]:
# Lineare Regression: Überleben in Bezug auf den Fahrpreis (in Pfund Sterling) mit beschränktem Modell
ols_restr <- lm(Survived ~ Fare + FR + IR, data = titanic_train)
summary(ols_restr)


Call:
lm(formula = Survived ~ Fare + FR + IR, data = titanic_train)

Residuals:
    Min      1Q  Median      3Q     Max 
-0.9807 -0.3362 -0.2949  0.5574  0.7230 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 0.2769748  0.0204676  13.532  < 2e-16 ***
Fare        0.0022770  0.0003287   6.927 8.26e-12 ***
FR          0.1400795  0.0419142   3.342 0.000866 ***
IR          0.0824057  0.0565821   1.456 0.145638    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.4678 on 887 degrees of freedom
Multiple R-squared:  0.07896,	Adjusted R-squared:  0.07585 
F-statistic: 25.35 on 3 and 887 DF,  p-value: 9.712e-16


Wie kann man die Koeffizienten interpretieren?

### Unrestringierte Regression

Nun interagieren wir mit dem gesamten Modell mit $Woman$ und kontrollieren zusätzlich für den Dummy $Woman$. Dadurch können differenzierte Überlebenschancen von Männern und Frauen ermittelt werden.

Wir beginnen mit der Generierung der Interaktionsterme.

In [7]:
# Generierung von Interaktionsvariablen
titanic_train$Wo_Fare <- titanic_train$Woman * titanic_train$Fare
titanic_train$Wo_FR <- titanic_train$Woman * titanic_train$FR
titanic_train$Wo_IR <- titanic_train$Woman * titanic_train$IR
print('Interaktionsterme generiert')

[1] "Interaktionsterme generiert"


Wir schätzen nun das unrestringierte Modell, das alle Interaktionsterme und den Geschlechtsdummy enthält. Es erlaubt also volle Flexibilität in Bezug auf das Geschlecht.

In [8]:
# Lineare Regression: Überleben in Bezug auf den Fahrpreis (in Pfund Sterling) mit unbeschränktem Modell
ols_unrestr <- lm(Survived ~ Fare + FR + IR + Woman + Wo_Fare + Wo_FR + Wo_IR, data = titanic_train)
summary(ols_unrestr)


Call:
lm(formula = Survived ~ Fare + FR + IR + Woman + Wo_Fare + Wo_FR + 
    Wo_IR, data = titanic_train)

Residuals:
    Min      1Q  Median      3Q     Max 
-0.8588 -0.1804 -0.1562  0.2572  0.9349 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.1458919  0.0208897   6.984 5.65e-12 ***
Fare         0.0013224  0.0003974   3.328 0.000912 ***
FR           0.0955509  0.0464049   2.059 0.039780 *  
IR          -0.0910213  0.0653052  -1.394 0.163734    
Woman        0.4889618  0.0384016  12.733  < 2e-16 ***
Wo_Fare      0.0001550  0.0005722   0.271 0.786545    
Wo_FR        0.0352544  0.0730651   0.483 0.629566    
Wo_IR        0.1875012  0.0979603   1.914 0.055936 .  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.3995 on 883 degrees of freedom
Multiple R-squared:  0.3312,	Adjusted R-squared:  0.3259 
F-statistic: 62.46 on 7 and 883 DF,  p-value: < 2.2e-16


Wie ist die statistische Signifikanz der Interaktionsterme zu interpretieren?

### Test auf Heterogenität

Die gemeinsame Signifikanz mehrerer Interaktionsterme kann mit Hilfe eines F-Tests überprüft werden. Die Teststatistik wird wie folgt berechnet,

$$F-Stat = \frac{SSR_{restr}-SSR_{unrestr}}{SSR_{unrestr}} \cdot\frac{\text{Nenner Freiheitsgrade}}{\text{Zähler Freiheitsgrade}},$$

wobei $SSR$ für "*Sum of Squared Residuals*" steht und wie folgt berechnet wird

$$SSR=\sum(\hat{y}-y)^2.$$ 

Vereinfacht ausgedrückt misst die F-Statistik die Wachstumsrate des $SSR$, wenn Variablen weggelassen werden, gewichtet mit dem Verhältnis der Freiheitsgrade.

Die Nullhypothese besagt, dass es keinen signifikanten Unterschied zwischen dem restringierten und dem unrestringierten Modell gibt.

Zunächst müssen wir also die $SSR$ für das restringierte und das unrestringierte Modell berechnen.

In [9]:
# Berechnung der Summe der quadratischen Residuen für beschränktes und unbeschränktes Modell
ssr_restr <- sum((fitted(ols_restr) - titanic_train$Survived)^2)
ssr_unrestr <- sum((fitted(ols_unrestr) - titanic_train$Survived)^2)

Als nächstes müssen wir die Freiheitsgrade bestimmen. Dazu müssen wir zunächst die Anzahl der Beobachtungen im Datensatz kennen.

In [10]:
# Anzahl an Beobachtungen im Datensatz
n_obs <- nrow(titanic_train)
n_obs

[1] 891

Die Anzahl der Freiheitsgrade entspricht der Differenz zwischen der Anzahl der Beobachtungen und der Anzahl der geschätzten Parameter. Im restringierten Modell haben wir 4 geschätzte Parameter.

In [11]:
# Freiheitsgrade für beschränktes Modell
df_restr <- ols_restr$df.residual
df_restr

[1] 887

Im unrestringierten Modell haben wir 8 geschätzte Parameter. Daraus koennen wir die Nenner Freiheitsgrade berechnen.

In [12]:
# Freiheitsgrade für unbeschränktes Modell
df_unrestr <- ols_unrestr$df.residual
df_unrestr # Nenner Freiheitsgrade

[1] 883

Die Zähler Freiheitsgrade sind die Differenz zwischen den Freiheitsgraden des restringierten und des unrestringierten Modells.

In [13]:
# Differenz der Freiheitsgrade
df_diff <- df_restr - df_unrestr
df_diff # Zähler Freiheitsgrade

[1] 4

Nun haben wir alle notwendigen Zutaten, um die F-Statistik mit Hilfe der obigen Formel zu berechnen.

In [14]:
# Berechnung der Teststatistik
test_stat <- (ssr_restr - ssr_unrestr) / ssr_unrestr * (df_unrestr) / df_diff
test_stat

[1] 83.24349

Schliesslich kann der kritische Wert aus dem oberen Quantil der F-Verteilung (hier 99%-Quantil für 1% Signifikanzniveau) bestimmt werden.

In [21]:
# Berechnung des kritischen Werts
F_crit <- qf(p = 0.99, df1 = df_diff, df2 = df_unrestr)
F_crit

[1] 3.34045

Kann die Nullhypothese aufgrund der Ergebnisse verworfen werden? Was bedeutet dies für die Überlebenschancen von Männern und Frauen? Können wir auch etwas darüber aussagen, ob es bei Frauen weniger Diskriminierung nach sozialer Schicht gab als bei Männern?